<a href="https://colab.research.google.com/github/meetAmarAtGithub/8_Fuzzy_Logic/blob/main/FinalExam_GA_Feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm for feature selection

Develop a GA-based feature selection data for the adult data set for the data provided. Compare your results with at least three ML methods (with and without feature selection). Draw your inferences and explain them with evidence. (https://www.kaggle.com/datasets/wenruliu/adult-income-dataset)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd "/content/gdrive/My Drive/Colab Notebooks/Reva/8_Fuzzy_Logic/FinalExam"

/content/gdrive/My Drive/Colab Notebooks/Reva/8_Fuzzy_Logic/FinalExam


In [1]:
!pip install sklearn-genetic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 4.0 MB/s 
     |████████████████████████████████| 132 kB 40.8 MB/s 


In [2]:
#Data management
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

#Modeling - AIML
from genetic_selection import GeneticSelectionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import ComplementNB

#Evaluation
from sklearn import metrics

In [21]:
#import pandas as pd
df = pd.read_csv("adult.csv")
df.head(5)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


# Data Management

In [22]:
#Dummy coding for categorical variables
for x in df.columns:
    if df[x].dtype == "object":
        df[x] =pd.Categorical(df[x]).codes

In [23]:
df.head(5)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,4,226802,1,7,4,7,3,2,1,0,0,40,39,0
1,38,4,89814,11,9,2,5,0,4,1,0,0,50,39,0
2,28,2,336951,7,12,2,11,0,4,1,0,0,40,39,1
3,44,4,160323,15,10,2,7,0,2,1,7688,0,40,39,1
4,18,0,103497,15,10,4,0,3,4,0,0,0,30,39,0


In [24]:
X = df.drop(['income'], axis=1)
y = df['income'].astype(str)
print("Number of features - ", len(X.columns))

Number of features -  14


In [25]:
df.sample(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
6233,37,4,126675,15,10,6,7,2,4,1,0,0,40,0,0
38842,28,4,37821,8,11,4,12,4,4,0,0,0,55,0,0
33981,32,6,115066,11,9,2,12,0,4,1,0,0,60,39,1
38948,39,4,216256,9,13,2,4,0,4,1,0,0,60,22,1
45987,31,5,113530,11,9,4,3,1,4,1,0,0,45,39,0
19053,20,4,436253,15,10,4,8,3,4,1,0,0,35,39,0
39321,40,4,51290,12,14,2,10,0,4,1,0,0,50,39,1
46991,35,4,187167,11,9,0,8,1,4,0,0,0,35,39,0
35865,64,6,192695,4,3,2,5,0,4,1,0,0,70,2,0
41867,33,4,176992,15,10,2,3,0,4,1,0,0,40,39,1


In [26]:
#Sample for training and testing the model
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.4, random_state=1)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((29305, 14), (19537, 14), (29305,), (19537,))

In [27]:
#Build base models
lr = LogisticRegression(random_state=0).fit(X_train, Y_train)
rf = RandomForestClassifier(n_estimators = 7).fit(X_train, Y_train)
knn = knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, Y_train)
svc = SVC().fit(X_train, Y_train)
nb = ComplementNB().fit(X_train, Y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
lr_pred = lr.predict(X_train)
rf_pred = rf.predict(X_train)
knn_pred = knn.predict(X_train)
svc_pred = svc.predict(X_train)
nb_pred = nb.predict(X_train)

print("Accuracy score of LR = ",100*metrics.accuracy_score(Y_train, lr_pred))
print("Accuracy score of RF = ",100*metrics.accuracy_score(Y_train, rf_pred))
print("Accuracy score of KNN = ",100*metrics.accuracy_score(Y_train, knn_pred))
print("Accuracy score of SVC = ",100*metrics.accuracy_score(Y_train, svc_pred))
print("Accuracy score of NB = ",100*metrics.accuracy_score(Y_train, nb_pred))

Accuracy score of LR =  79.38918273332196
Accuracy score of RF =  98.53267360518683
Accuracy score of KNN =  86.5040095546835
Accuracy score of SVC =  79.73724620371951
Accuracy score of NB =  78.37911619177615


In [29]:
estimator = DecisionTreeClassifier()
fs_model = GeneticSelectionCV(
    estimator, cv=5, verbose=0,
    scoring="accuracy", max_features=5,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
fs_model = fs_model.fit(X, y)
features =  X.columns[fs_model.support_]
print('Features:', features)

Features: Index(['educational-num', 'occupation', 'relationship', 'capital-gain',
       'capital-loss'],
      dtype='object')


In [31]:
#Build optimized models
X =  df[features]
y = df['income'].astype(float)
print("Number of features - ", len(X.columns))

Number of features -  5


In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.4, random_state=1)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((29305, 5), (19537, 5), (29305,), (19537,))

In [33]:
lr = LogisticRegression(random_state=0).fit(X_train, Y_train)
rf = RandomForestClassifier(n_estimators = 7).fit(X_train, Y_train)
knn = knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, Y_train)
svc = SVC().fit(X_train, Y_train)
nb = ComplementNB().fit(X_train, Y_train)

In [34]:
lr_pred = lr.predict(X_train)
rf_pred = rf.predict(X_train)
knn_pred = knn.predict(X_train)
svc_pred = svc.predict(X_train)
nb_pred = nb.predict(X_train)

print("Accuracy score of LR = ",100*metrics.accuracy_score(Y_train, lr_pred))
print("Accuracy score of RF = ",100*metrics.accuracy_score(Y_train, rf_pred))
print("Accuracy score of KNN = ",100*metrics.accuracy_score(Y_train, knn_pred))
print("Accuracy score of SVC = ",100*metrics.accuracy_score(Y_train, svc_pred))
print("Accuracy score of NB = ",100*metrics.accuracy_score(Y_train, nb_pred))

Accuracy score of LR =  81.98942160040949
Accuracy score of RF =  86.99880566456237
Accuracy score of KNN =  84.52482511516806
Accuracy score of SVC =  80.28322811806858
Accuracy score of NB =  78.04470226923733


#Conclusion :

## With 14 features using Random Forest we have achieved 98.5% accuracy, but using 5 features and using Genetic Algorithm we are able to achieve 86.99% accuracy or, features - 'educational-num', 'occupation', 'relationship', 'capital-gain', 'capital-loss' able to predict income with almost 87% accuracy. 